# ***Step-1: Setup & Import Libraries***

In [ ]:
import torch
import torch.quantization
import torch.nn as nn
import copy
import os
import time

In [ ]:
import numpy as np

# ***Step-2: Float Tensor Representation from Netron***

*** ## Reference of Netron:*** https://netron.app/

In [ ]:
# Here we can put tensors from any trained model (By using NETRON software)

x = torch.tensor([[ 0.8237,  0.5781,  0.6879],
        [ 0.3816,  0.7249,  0.0998]])

# x = torch.rand(m=2,n=3)

torch.save(x,'float_tensor.pt')
print('Float tensor (bytes)',os.path.getsize('float_tensor.pt'))


Float tensor (bytes) 747


***--Maximum Value and minimum value of x***

In [ ]:
# Min-Max value of float_32 tensor (x) find out for scale (s) and zero point (z)

b = torch.max(x)
print(b)

tensor(0.8237)


In [ ]:
a = torch.min(x)
print(a)

tensor(0.0998)


In [ ]:
# scale value

s = (b-a)/15

print(s)

tensor(0.0483)


In [ ]:
# zero point

z = torch.round(-a*15/(b-a))

print(z)

tensor(-2.)


# ***Step-3: Apply Quantization by round and clipping function (Affine mapping)***

In [ ]:
f = torch.round(x/s + z)
print(f)
xq = torch.clip(f, min=0, max=15) # Here min & max value we can change as per Tbit.
# But, I have checked for 4 bit
print (xq)
torch.save(xq,'qtz_tensor.pt')
print('Quantized tensor (bytes)',os.path.getsize('qtz_tensor.pt'))

tensor([[15., 10., 12.],
        [ 6., 13.,  0.]])
tensor([[15., 10., 12.],
        [ 6., 13.,  0.]])
Quantized tensor (bytes) 747


# ***Step-4: Apply Dequantization***

In [ ]:
xdq = s*(xq - z)
print(xdq)

tensor([[0.8204, 0.5791, 0.6756],
        [0.3861, 0.7239, 0.0965]])


In [ ]:
# xdq = xq.dequantize()
# print(xdq)

# ***Step-5: MAE/MSE loss between x and xdq***

***I. MAE loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn
# print input and target tensors
print("Input Tensor:\n", x)
print("Target Tensor:\n", xdq)
# create a criterion to measure the mean absolute error
mae = nn.L1Loss()
# compute the loss (mean absolute error)
output = mae(x, xdq)
# output.backward()
print("MAE loss:", output)

Input Tensor:
 tensor([[0.8237, 0.5781, 0.6879],
        [0.3816, 0.7249, 0.0998]])
Target Tensor:
 tensor([[0.8204, 0.5791, 0.6756],
        [0.3861, 0.7239, 0.0965]])
MAE loss: tensor(0.0042)


***II. MSE Loss***

In [ ]:
# Import the required libraries
import torch
import torch.nn as nn

# print input and target tensors
print("Input Tensor:\n", x)
print("Target Tensor:\n", xdq)

# create a criterion to measure the mean squared error
mse = nn.MSELoss()

# compute the loss (mean squared error)
output = mse(x, xdq)

# output.backward()
print("MSE loss:", output)

Input Tensor:
 tensor([[0.8237, 0.5781, 0.6879],
        [0.3816, 0.7249, 0.0998]])
Target Tensor:
 tensor([[0.8204, 0.5791, 0.6756],
        [0.3861, 0.7239, 0.0965]])
MSE loss: tensor(3.2323e-05)


## ***Step-6: MMSE Algorithm Performance***


In [ ]:
Qt = torch.transpose(xq, 0, 1)
print(Qt)

tensor([[15.,  6.],
        [10., 13.],
        [12.,  0.]])


In [ ]:
t = torch.mm(Qt, xq)
print(t)

tensor([[261., 228., 180.],
        [228., 269., 120.],
        [180., 120., 144.]])


In [ ]:
T = torch.inverse(t)
print(T)

tensor([[ 470691.6875, -217242.3125, -407329.3438],
        [-217242.2656,  100265.6719,  187998.1094],
        [-407329.3750,  187998.1719,  352496.5938]])


In [ ]:
p = torch.mm(T, Qt)
print(p)

tensor([[0.3750, 0.0000],
        [0.0625, 0.1250],
        [0.1250, 0.0000]])


In [ ]:
alpha = torch.mm(p, x)
print(alpha)

tensor([[0.3089, 0.2168, 0.2580],
        [0.0992, 0.1267, 0.0555],
        [0.1030, 0.0723, 0.0860]])


In [ ]:
j = torch.mm(xq, alpha)
print(j)


tensor([[6.8607, 5.3864, 5.4560],
        [3.1427, 2.9484, 2.2689]])
